# Deliverable 2: X-ray Projection & CT Imaging

**Due:** Sunday, February 23, 2026 at 11:59 PM

---

## Student Information

**Name:** [Your Name]  
**Student ID:** [Your ID]  
**Date:** [Submission Date]

---

## Setup and Imports

Run this cell first to import necessary libraries and load data files.

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy.fft import fft, ifft, fftfreq

# Configure plotting
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12
%matplotlib inline

# Import helper utilities
import sys
sys.path.insert(0, 'data')
from ct_utils import (
    create_shepp_logan_phantom,
    forward_project,
    create_ramp_filter,
    calculate_rmse,
    calculate_ssim,
    mu_to_hu,
    hu_to_mu
)

print("All imports successful!")

In [ ]:
# Load data files
projection_scenarios = pd.read_csv('data/projection_scenarios.csv')
focal_spot_data = pd.read_csv('data/focal_spot_data.csv')
hu_calibration = pd.read_csv('data/hu_calibration.csv')
phantom_inserts = pd.read_csv('data/phantom_inserts.csv')

# Load phantom and sinograms
shepp_logan = np.load('data/shepp_logan_phantom.npy')
sinogram_180 = np.load('data/sinogram_180.npy')
sinogram_90 = np.load('data/sinogram_90.npy')
sinogram_45 = np.load('data/sinogram_45.npy')

# Load angles
angles_180 = np.load('data/projection_angles_180.npy')
angles_90 = np.load('data/projection_angles_90.npy')
angles_45 = np.load('data/projection_angles_45.npy')

# Load CT phantoms for artifact analysis
ct_clean = np.load('data/ct_phantom_clean.npy')
ct_beam_hard = np.load('data/ct_phantom_beam_hardening.npy')
ct_ring = np.load('data/ct_phantom_ring.npy')
ct_combined = np.load('data/ct_phantom_combined_artifacts.npy')

print("Data loaded successfully!")
print(f"\nShepp-Logan phantom shape: {shepp_logan.shape}")
print(f"Sinogram shapes: 180={sinogram_180.shape}, 90={sinogram_90.shape}, 45={sinogram_45.shape}")
print(f"\nClinical scenarios available: {len(projection_scenarios)}")
print(projection_scenarios[['scenario_id', 'name']].to_string(index=False))

---

# Part 1: Projection Geometry Optimizer (30%)

## Background

In projection radiography, image sharpness depends on geometric factors. The key equations are:

**Magnification:**
$$m = \frac{d_{FID}}{d_{FID} - d_{OID}}$$

**Geometric Unsharpness:**
$$U_g = X_f \cdot \frac{m - 1}{m}$$

**Total Unsharpness:**
$$U_{total} = \sqrt{\left(\frac{U_r}{m}\right)^2 + U_g^2}$$

**Inverse Square Law (mAs adjustment):**
$$mAs_2 = mAs_1 \cdot \left(\frac{d_{FID,2}}{d_{FID,1}}\right)^2$$

Where:
- $d_{FID}$ = Focus-to-Image Distance (SID)
- $d_{OID}$ = Object-to-Image Distance
- $X_f$ = Focal spot size
- $U_r$ = Receptor blur
- $U_g$ = Geometric unsharpness

In [ ]:
# Explore the clinical scenarios
print("Clinical Projection Scenarios:")
print("="*80)
display(projection_scenarios)

print("\nFocal Spot Options:")
print("="*80)
display(focal_spot_data)

## 1.1 Implement Geometric Calculations

Create functions to calculate magnification, geometric unsharpness, total unsharpness, and required mAs.

In [ ]:
def calculate_magnification(fid_cm: float, oid_cm: float) -> float:
    """
    Calculate magnification factor.
    
    Parameters
    ----------
    fid_cm : float
        Focus-to-image distance in cm (SID)
    oid_cm : float
        Object-to-image distance in cm
        
    Returns
    -------
    m : float
        Magnification factor (>= 1)
    """
    # YOUR CODE HERE
    pass


def calculate_geometric_unsharpness(focal_spot_mm: float, 
                                     magnification: float) -> float:
    """
    Calculate geometric unsharpness (penumbra width).
    
    Parameters
    ----------
    focal_spot_mm : float
        Focal spot size in mm
    magnification : float
        Magnification factor
        
    Returns
    -------
    Ug : float
        Geometric unsharpness in mm
    """
    # YOUR CODE HERE
    pass


def calculate_total_unsharpness(receptor_blur_mm: float,
                                 geometric_unsharpness_mm: float,
                                 magnification: float) -> float:
    """
    Calculate total system unsharpness.
    
    Parameters
    ----------
    receptor_blur_mm : float
        Receptor blur in mm
    geometric_unsharpness_mm : float
        Geometric unsharpness in mm
    magnification : float
        Magnification factor
        
    Returns
    -------
    U_total : float
        Total unsharpness in mm
    """
    # YOUR CODE HERE
    pass


def calculate_required_mAs(reference_mAs: float,
                           reference_fid_cm: float,
                           new_fid_cm: float) -> float:
    """
    Calculate required mAs for new FID using inverse square law.
    
    Parameters
    ----------
    reference_mAs : float
        mAs at reference distance
    reference_fid_cm : float
        Reference FID in cm
    new_fid_cm : float
        New FID in cm
        
    Returns
    -------
    new_mAs : float
        Required mAs at new distance
    """
    # YOUR CODE HERE
    pass

In [ ]:
# Test your functions with a simple example
# Expected: m = 1.25 for FID=100cm, OID=20cm

test_fid = 100  # cm
test_oid = 20   # cm
test_focal = 0.3  # mm (fine focus)
test_receptor_blur = 0.15  # mm

m = calculate_magnification(test_fid, test_oid)
Ug = calculate_geometric_unsharpness(test_focal, m)
U_total = calculate_total_unsharpness(test_receptor_blur, Ug, m)

print(f"Test Results:")
print(f"  Magnification: {m:.3f}")
print(f"  Geometric unsharpness: {Ug:.4f} mm")
print(f"  Total unsharpness: {U_total:.4f} mm")

## 1.2 Design the Geometry Optimizer

Create an algorithm that finds the optimal SID (FID) and focal spot selection for a given clinical scenario.

In [ ]:
def optimize_projection_geometry(body_thickness_cm: float,
                                  object_depth_cm: float,
                                  max_unsharpness_mm: float,
                                  receptor_blur_mm: float,
                                  max_mAs: float,
                                  reference_mAs: float = 10.0,
                                  reference_fid: float = 100.0,
                                  fid_range: tuple = (50, 250),
                                  focal_spots: list = [0.3, 1.0]):
    """
    Find optimal projection geometry for a clinical scenario.
    
    Parameters
    ----------
    body_thickness_cm : float
        Total thickness of body part
    object_depth_cm : float
        Depth of structure of interest from receptor
    max_unsharpness_mm : float
        Maximum acceptable total unsharpness
    receptor_blur_mm : float
        Inherent receptor blur
    max_mAs : float
        Maximum tube loading constraint
    reference_mAs : float
        Reference mAs at reference_fid
    reference_fid : float
        Reference FID for mAs calculation
    fid_range : tuple
        (min, max) FID to consider in cm
    focal_spots : list
        Available focal spot sizes in mm
        
    Returns
    -------
    results : dict
        Dictionary containing optimal parameters and trade-off data
    """
    # YOUR CODE HERE
    # Hints:
    # 1. Create a grid of FID values to evaluate
    # 2. For each FID, calculate OID = object_depth_cm (receptor at skin)
    # 3. Calculate m, Ug, U_total, and required mAs for each focal spot
    # 4. Find configurations that meet constraints (U_total < max, mAs < max)
    # 5. Among valid configurations, find the one with minimum unsharpness
    #    or best balance of factors
    
    results = {
        'optimal_fid': None,
        'optimal_focal_spot': None,
        'optimal_magnification': None,
        'optimal_unsharpness': None,
        'optimal_mAs': None,
        'tradeoff_data': None  # For visualization
    }
    
    return results

In [ ]:
# Test your optimizer with the first scenario (Chest PA)
scenario = projection_scenarios.iloc[0]
print(f"Optimizing for: {scenario['name']}")
print(f"Body thickness: {scenario['body_part_thickness_cm']} cm")
print(f"Object depth: {scenario['object_depth_cm']} cm")

# Convert resolution requirement to unsharpness limit
# Spatial resolution (lp/mm) relates to smallest resolvable detail
# Unsharpness limit ≈ 1 / (2 × resolution)
max_unsharpness = 1.0 / (2 * scenario['min_resolution_lp_mm'])
print(f"Max unsharpness: {max_unsharpness:.3f} mm")

# Run optimization
# results = optimize_projection_geometry(
#     body_thickness_cm=scenario['body_part_thickness_cm'],
#     object_depth_cm=scenario['object_depth_cm'],
#     max_unsharpness_mm=max_unsharpness,
#     receptor_blur_mm=scenario['receptor_blur_mm'],
#     max_mAs=scenario['max_mAs']
# )
# print(f"\nOptimal Configuration:")
# print(f"  FID: {results['optimal_fid']} cm")
# print(f"  Focal spot: {results['optimal_focal_spot']} mm")

## 1.3 Trade-off Visualization

Create visualizations showing how unsharpness and mAs vary with SID for different focal spot sizes.

In [ ]:
# YOUR CODE HERE
# Create a figure showing:
# 1. Unsharpness vs FID for fine and broad focal spots
# 2. Required mAs vs FID
# 3. Perhaps a combined trade-off plot

# Suggested: Use subplots or a dual-axis plot

## 1.4 Clinical Scenario Analysis

Analyze at least 3 clinical scenarios from the provided data. For each, report:
- Optimal geometry
- Why fine vs. broad focal spot is preferred
- Any trade-offs or limitations

In [ ]:
# Analyze multiple scenarios
# YOUR CODE HERE

### Discussion: Part 1

*Write your analysis of the trade-offs between focal spot size, SID, unsharpness, and tube loading. When would you choose fine vs. broad focal spot?*

[YOUR DISCUSSION HERE]

---

# Part 2: CT Reconstruction Pipeline (30%)

## Background

Filtered Back Projection (FBP) reconstructs images from projections:

1. **Forward Problem:** Sinogram is the Radon transform of the image
2. **Filtering:** Apply ramp filter $|f|$ in frequency domain
3. **Back Projection:** Smear filtered projections back across image

The central slice theorem connects projections to the image's Fourier transform.

In [ ]:
# Visualize the phantom and sinogram
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(shepp_logan, cmap='gray')
axes[0].set_title('Shepp-Logan Phantom')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')

axes[1].imshow(sinogram_180, cmap='gray', aspect='auto')
axes[1].set_title('Sinogram (180 projections)')
axes[1].set_xlabel('Detector position')
axes[1].set_ylabel('Angle index')

axes[2].imshow(sinogram_45, cmap='gray', aspect='auto')
axes[2].set_title('Sinogram (45 projections)')
axes[2].set_xlabel('Detector position')
axes[2].set_ylabel('Angle index')

plt.tight_layout()
plt.show()

## 2.1 Implement Reconstruction Filters

Create functions for the ramp filter and windowed versions.

In [ ]:
def create_reconstruction_filter(n_detectors: int, 
                                  filter_type: str = 'ramp') -> np.ndarray:
    """
    Create a frequency-domain filter for FBP reconstruction.
    
    Parameters
    ----------
    n_detectors : int
        Number of detector elements (projection length)
    filter_type : str
        'ramp' - Standard ramp filter |f|
        'hamming' - Ramp with Hamming window (smoother)
        
    Returns
    -------
    H : ndarray
        Filter in frequency domain
    """
    # YOUR CODE HERE
    # Hints:
    # 1. Create frequency axis using np.fft.fftfreq(n_detectors)
    # 2. Ramp filter is |frequency|
    # 3. For Hamming: multiply by 0.54 + 0.46*cos(pi*f/f_max)
    pass

In [ ]:
# Visualize your filters
n = 256
# ramp = create_reconstruction_filter(n, 'ramp')
# hamming = create_reconstruction_filter(n, 'hamming')

# plt.figure(figsize=(10, 4))
# freq = np.fft.fftfreq(n)
# plt.plot(np.fft.fftshift(freq), np.fft.fftshift(ramp), label='Ramp')
# plt.plot(np.fft.fftshift(freq), np.fft.fftshift(hamming), label='Hamming')
# plt.xlabel('Frequency')
# plt.ylabel('Filter magnitude')
# plt.title('Reconstruction Filters')
# plt.legend()
# plt.grid(True)
# plt.show()

## 2.2 Implement Filtered Back Projection

Create the core FBP reconstruction algorithm.

In [ ]:
def filtered_back_projection(sinogram: np.ndarray,
                              angles: np.ndarray,
                              filter_type: str = 'ramp',
                              output_size: int = None) -> np.ndarray:
    """
    Reconstruct an image from its sinogram using filtered back projection.
    
    Parameters
    ----------
    sinogram : ndarray
        2D array where each row is a projection at the corresponding angle
        Shape: (n_angles, n_detectors)
    angles : ndarray
        Projection angles in degrees
    filter_type : str
        Type of reconstruction filter ('ramp' or 'hamming')
    output_size : int
        Size of output image (default: n_detectors)
        
    Returns
    -------
    reconstruction : ndarray
        Reconstructed 2D image
    """
    # YOUR CODE HERE
    # Algorithm:
    # 1. Get dimensions
    # 2. Create the filter
    # 3. Initialize output image
    # 4. For each projection:
    #    a. Take FFT of projection
    #    b. Multiply by filter
    #    c. Take inverse FFT
    #    d. Back-project: rotate the filtered projection and add to image
    # 5. Normalize by number of projections
    
    pass

In [ ]:
# Test your FBP implementation
# reconstruction = filtered_back_projection(sinogram_180, angles_180, 'ramp')

# fig, axes = plt.subplots(1, 3, figsize=(15, 5))
# axes[0].imshow(shepp_logan, cmap='gray')
# axes[0].set_title('Original Phantom')

# axes[1].imshow(reconstruction, cmap='gray')
# axes[1].set_title('FBP Reconstruction (180 proj)')

# diff = shepp_logan - reconstruction
# axes[2].imshow(diff, cmap='RdBu', vmin=-0.5, vmax=0.5)
# axes[2].set_title('Difference')

# plt.tight_layout()
# plt.show()

## 2.3 Compare Reconstruction Quality

Analyze how reconstruction quality varies with:
- Number of projections (180, 90, 45)
- Filter choice (ramp vs. Hamming)

In [ ]:
# YOUR CODE HERE
# 1. Reconstruct images with different parameters
# 2. Calculate RMSE and/or SSIM vs. ground truth
# 3. Create comparison figure

In [ ]:
# Quantitative comparison table
# YOUR CODE HERE

### Discussion: Part 2

*Analyze the trade-off between resolution and noise for different filters. Why does reducing the number of projections cause streak artifacts? How does this relate to the Nyquist sampling theorem?*

[YOUR DISCUSSION HERE]

---

# Part 3: Hounsfield Unit Analysis & Artifact Investigation (30%)

## Background

Hounsfield units (HU) standardize CT attenuation values:

$$HU = 1000 \times \frac{\mu_{tissue} - \mu_{water}}{\mu_{water}}$$

Reference values:
- Air: -1000 HU
- Water: 0 HU  
- Bone: ~+1000 HU

In [ ]:
# Visualize the CT phantoms
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

images = [ct_clean, ct_beam_hard, ct_ring, ct_combined]
titles = ['Clean Phantom', 'Beam Hardening', 'Ring Artifacts', 'Combined']

for ax, img, title in zip(axes.flat, images, titles):
    im = ax.imshow(img, cmap='gray')
    ax.set_title(title)
    plt.colorbar(im, ax=ax, label='μ (mm⁻¹)')

plt.tight_layout()
plt.show()

# Display insert information
print("\nPhantom Insert Information:")
display(phantom_inserts)

## 3.1 HU Calibration and Measurement

Measure HU values in the clean phantom and compare to expected values.

In [ ]:
def measure_hu_in_roi(image: np.ndarray, 
                      center_x: int, center_y: int, 
                      radius: int,
                      mu_water: float = 0.019) -> dict:
    """
    Measure HU statistics in a circular region of interest.
    
    Parameters
    ----------
    image : ndarray
        CT image with linear attenuation values
    center_x, center_y : int
        Center of ROI
    radius : int
        Radius of circular ROI
    mu_water : float
        Reference μ for water
        
    Returns
    -------
    results : dict
        Dictionary with mean_hu, std_hu, mean_mu, std_mu
    """
    # YOUR CODE HERE
    pass

In [ ]:
# Measure HU values for each insert
# YOUR CODE HERE

# Compare to expected values from hu_calibration
print("\nExpected HU Values:")
display(hu_calibration)

## 3.2 Artifact Detection Algorithms

Design algorithms to automatically detect:
1. **Beam hardening** (cupping artifact)
2. **Ring artifacts**

In [ ]:
def detect_beam_hardening(image: np.ndarray) -> dict:
    """
    Detect beam hardening (cupping) artifact.
    
    Beam hardening causes HU values to be lower in the center
    of homogeneous regions compared to the edges.
    
    Parameters
    ----------
    image : ndarray
        CT image to analyze
        
    Returns
    -------
    results : dict
        Detection results including:
        - detected: bool
        - severity: float (0-1 scale)
        - radial_profile: ndarray
    """
    # YOUR CODE HERE
    # Hints:
    # 1. Calculate radial profile from image center
    # 2. In a uniform region, cupping shows as lower values at center
    # 3. Measure the "dip" in the profile
    pass


def detect_ring_artifact(image: np.ndarray) -> dict:
    """
    Detect ring artifacts in CT image.
    
    Ring artifacts appear as concentric circles centered
    on the rotation axis.
    
    Parameters
    ----------
    image : ndarray
        CT image to analyze
        
    Returns
    -------
    results : dict
        Detection results including:
        - detected: bool
        - severity: float
        - ring_radii: list of detected ring positions
    """
    # YOUR CODE HERE
    # Hints:
    # 1. Calculate radial profile
    # 2. Look for peaks/valleys in the profile that don't correspond
    #    to expected structure
    # 3. Rings show as periodic oscillations in radial profile
    pass

In [ ]:
# Test your artifact detection
# YOUR CODE HERE

## 3.3 Beam Hardening Correction

Implement a polynomial correction for beam hardening.

In [ ]:
def correct_beam_hardening(image: np.ndarray, 
                           order: int = 2) -> np.ndarray:
    """
    Apply polynomial beam hardening correction.
    
    Parameters
    ----------
    image : ndarray
        CT image with beam hardening artifact
    order : int
        Order of polynomial correction
        
    Returns
    -------
    corrected : ndarray
        Beam hardening corrected image
    """
    # YOUR CODE HERE
    # Hints:
    # 1. Calculate radial distance map from center
    # 2. Estimate correction profile (polynomial fit to cupping)
    # 3. Add correction to image
    pass

In [ ]:
# Apply correction and compare
# YOUR CODE HERE

# Show before/after comparison
# Calculate improvement in HU accuracy

### Discussion: Part 3

*Discuss the physical causes of beam hardening and ring artifacts. Why is beam hardening correction challenging? What are the limitations of your correction approach?*

[YOUR DISCUSSION HERE]

---

# Part 4: Integration Memo (10%)

## Clinical Question

> *"A radiologist asks: For evaluating a patient with suspected lung nodules, when should I order a chest X-ray versus a chest CT? Discuss the trade-offs in terms of projection geometry, dose, contrast resolution, and spatial resolution. Use quantitative reasoning from your analyses in Parts 1-3."*

Write your technical memo (300-500 words) below.

## Technical Memo: Chest X-ray vs. CT for Lung Nodule Evaluation

**To:** Radiology Department  
**From:** [Your Name]  
**Date:** [Date]  
**Re:** Imaging Modality Selection for Suspected Lung Nodules

---

[YOUR MEMO HERE - 300-500 words]

Address:
1. Geometric considerations (superimposition in projection vs. tomographic sectioning)
2. Contrast resolution (why CT is better for low-contrast lesions)
3. Spatial resolution comparison
4. Radiation dose trade-offs
5. Your recommendation with quantitative justification from Parts 1-3

---

---

# Summary and Conclusions

*Provide a brief summary of your key findings from each part and what you learned.*

[YOUR SUMMARY HERE]

---

## References

*List any external resources, code references, or materials you consulted.*

1. IAEA Diagnostic Radiology Physics Handbook, Chapters 5, 6, 7, 11
2. [Add your references here]